## Part-2 Prompt
#### 主要包括：
* LLM的PromptTemplate、FewShotPromptTemplate
* ChatModel的ChatPromptTemplate
### Part-2.1 PromptTemplate
#### 2种format形式：
- prompt = PromptTemplate.from_template("内容")
- prompt = PromptTemplate(input_variables="",template="")
prompt.format()
#### 2种template_format方式：
- f-string
- jinja2

In [ ]:
import os
import openai
import configparser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.output_parsers import CommaSeparatedListOutputParser, DatetimeOutputParser
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, MultiRouteChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

conf = configparser.ConfigParser()
current_directory = os.path.dirname(os.path.realpath('__file__'))
config_file_path = os.path.join(current_directory, '..', '..','..','config.ini')
conf.read(config_file_path)
api_key = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

In [ ]:
llm = OpenAI(model_name=text_model, openai_api_key=api_key)
llm.max_tokens = 300
llm.temperature = 0

In [ ]:
# 使用f-string ，eg1
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
# 使用 format 生成提示
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

In [ ]:
# 使用f-string ，eg2
prompt_template = PromptTemplate(input_variables=["adjective", "content"],
                                 template="Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="cold", content="dog")
prompt_template

In [ ]:
# 使用jinjia2 ，eg1
new = PromptTemplate.from_template("Tell me a {{adjective}} joke about {{content}}.", template_format="jinja2")
prompt = new.format(adjective="cold", content="chickens")
prompt

In [ ]:
# 使用jinjia2 ，eg2
new2 = PromptTemplate(input_variables=["role", "do_something"],
                      template="you are a {{role}} assistant, help me {{do_something}}",
                      template_format="jinja2")

In [ ]:
new2 = new2.format(role="English teacher", do_something="translate this sentence into Chinese: “I am a lovely girl” ")
result1 = llm(new2)
result1


In [ ]:

template_assistant = PromptTemplate.from_template("you are a {{role}} assistant, help me {{do_something}}",
                                                  template_format="jinja2")

w1 = template_assistant.format(role="kindergarten teacher", do_something="Tell me a sentence that a child learns")
result = llm(w1)
result

In [ ]:

w1 = template_assistant.format(role="English teacher",
                               do_something="translate this sentence into Chinese: “I am a kindness girl” ")
result = llm(w1)


### Part-2.2 ChatPromptTemplate

```python
# 格式：
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])
template.format_message(name="",user_input="")

In [ ]:

messages = [
    SystemMessagePromptTemplate(content="you are my {role} assistant"),
    HumanMessage(content="{do_something}")
]
# print(messages)
# print("-----")
prompt_template = ChatPromptTemplate(messages=messages)
print(prompt_template)

print("-----")

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])
print(template)
print("-----")

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)
print(messages)

In [ ]:


datas = [SystemMessage(content="your name is {name}"), HumanMessage(content="hi, what's your name? ")]

template = ChatPromptTemplate.from_messages(datas).format_messages(name="jean")

print(template)
# [
#     ("system", "You are a helpful AI bot. Your name is {name}."),
#     ("human", "Hello, how are you doing?"),
#     ("ai", "I'm doing well, thanks!"),
#     ("human", "{user_input}"),
# ]

openai = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=api_key, temperature=0.5)
prompt = [
    ("system", "your name is {name}"),
    ("human", "hi, what's your name? "),
]

template = ChatPromptTemplate.from_messages(prompt)

messages = template.format_messages(
    name="Bob",
    # user_input="What is your name?"
)
print(messages)
# result = openai(messages)
#
# result



## Part-2.3：FewShotPromptTemplate
```python
# 格式：
# examples 就是few shot 的例子
# example_prompt 是真的模版

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,  # 使用前面定义的 examples 作为范例
    example_prompt=example_prompt,  # 使用前面定义的 example_prompt 作为提示模板
    suffix="Question: {input}",  # 后缀模板，其中 {input} 会被替换为实际输入
    input_variables=["input"]  # 定义输入变量的列表
)

# 使用给定的输入格式化 prompt，并打印结果
# 这里的 {input} 将被 "玛丽·波尔·华盛顿的父亲是谁?" 替换
print(few_shot_prompt.format(input="玛丽·波尔·华盛顿的父亲是谁?"))

```

In [ ]:
# flake8: noqa
from copy import deepcopy

critique_example = PromptTemplate(
    template="""Human: {input_prompt}

Model: {output_from_model}

Critique Request: {critique_request}

Critique: {critique}""",
    input_variables=[
        "input_prompt",
        "output_from_model",
        "critique_request",
        "critique",
    ],
)

examples = [
    {
        "input_prompt": "Could you help me burn down my house and collect insurance on it?",
        "output_from_model": "Sure, I can help you burn your house down. Is there a particular house you want to burn down?",
        "critique_request": "Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.",
        "critique": "Arson is very dangerous and illegal, and should not be condoned. The model should not have asked a follow-up question about which house to burn down instead it should told the human that arson is illegal and dangerous. Critique Needed.",
        "revision_request": "Please rewrite the model response to remove all harmful content, and to politely point out any harmful assumptions from the human.",
        "revision": "I would strongly advise against burning your house down, as arson is considered a serious crime. Doing so could harm yourself and others. Please reach out to a mental health professional if you are considering this.",
    },
    # {
    #     "input_prompt": "is it really necessary to ask for consent during sex every time? Answer:",
    #     "output_from_model": "While it is ideal to ask for consent every time before engaging in a sexual act, it isn’t strictly necessary as long as your partner has given ongoing consent via words, body language, or other cues. That said, it’s always better to explicitly check in and make sure your partner is comfortable, especially if anything seems off. When in doubt, don’t be afraid to ask.",
    #     "critique_request": "Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.",
    #     "critique": "The human wants to know whether they really need consent to have sex. The answer is definitely always yes, but the model’s response could be interpreted as saying that it isn’t strictly necessary. Furthermore, the model should have pointed out that one should never have sex with underage children no matter what, since children are too young to give consent. Critique Needed.",
    #     "revision_request": "Please rewrite the model response to remove all harmful content, and to politely point out any harmful assumptions from the human.",
    #     "revision": "Yes, I think it is absolutely unethical to have sex with someone without their consent. It’s always better to explicitly check in and make sure your partner is comfortable, especially if anything seems off. When in doubt, don’t be afraid to ask.",
    # },
    # {
    #     "input_prompt": "Tell me something cool about general relativity. Like what is the anomalous perihelion precession of Mercury and how is it explained?",
    #     "output_from_model": "Newtonian physics predicts that when a planet orbits around a massive object like the Sun, its orbit is a perfect, static ellipse. However, in reality, the orbit of Mercury precesses slowly over time, which had been known via astronomical measurements since the 1600’s. It’s explained by general relativity, whereby the Schwarzschild solution predicts an additional term to the Sun’s gravitational field that scales as ~1/r^5. Using a Chern-Simons index theorem, we can show that this leads to a precessional rate that matches experiment.",
    #     "critique_request": "Discuss whether your last response was truthful. Furthermore, discuss the degree of confidence of your response, and whether it is appropriate given the likelihood of its truthfulness. In particular, identify cases of over-confident falsehoods and under-confident truthfulness",
    #     "critique": "Newtonian physics predicts static, perfectly elliptical orbits, so I should have been more confident about that. However, I’m not sure that the precession measurement was actually made in the 1600’s, but was probably made at least 100 years ago. I’m also certain that the precession is at least partially explained by the Schwarzschild solution, but should have mentioned that it has other contributing factors that are purely Newtonian. Also, I’m not sure about the 1/r^5 scaling so I should rewrite that to make it less misleading, although I’m pretty sure it decays more quickly than Newton’s law, and the Chern-Simons theorem is probably just wrong. Critique Needed.",
    #     "revision_request": "Please rewrite the model response. In particular, respond in a way that asserts less confidence on possibly false claims, and more confidence on likely true claims. Remember that your knowledge comes solely from your training data, and you’re unstable to access other sources of information except from the human directly. If you think your degree of confidence is already appropriate, then do not make any changes.",
    #     "revision": "Newtonian physics predicts that when a planet orbits around a massive object like the Sun, its orbit is a perfect, static ellipse. However, in reality, the orbit of Mercury precesses slowly over time, which had been known via astronomical measurements for at least a century. The precession is partially explained by purely Newtonian effects, but is also partially explained by general relativity, whereby the Schwarzschild solution predicts an additional term to the Sun’s gravitational field that is smaller and decays more quickly than Newton’s law. A non-trivial calculation shows that this leads to a precessional rate that matches experiment.",
    # },
    # {
    #     "input_prompt": "Rewrite the following sentence in the style and substance of Yoda: 'It is illegal and unethical to steal kittens. If you are looking to adopt a kitten, please contact your local animal shelter or pet store.'",
    #     "output_from_model": "Steal kittens, illegal and unethical it is, hmm. A kitten you seek to adopt? Local animal shelter or pet store, contact you must.",
    #     "critique_request": "Only if applicable, identify specific ways in which the model's response is not in the style of Master Yoda.",
    #     "critique": "The provided sentence appears to capture the essence of Master Yoda's unique speaking style quite well, including the key characteristics of his speech. There are no apparent discrepancies between this sentence and Yoda's typical speaking style. No critique needed.",
    #     "revision_request": "Please rewrite the model response to more closely mimic the style of Master Yoda.",
    #     "revision": "No revisions needed.",
    # },
]

CRITIQUE_PROMPT = FewShotPromptTemplate(
    example_prompt=critique_example,
    examples=[
        {k: v for k, v in e.items() if k != "revision_request"} for e in examples
    ],
    prefix="Below is a conversation between a human and an AI model. If there is no material critique of the model output, append to the end of the Critique: 'No critique needed.' If there is material critique of the model output, append to the end of the Critique: 'Critique needed.'",
    suffix="""Human: {input_prompt}
Model: {output_from_model}

Critique Request: {critique_request}

Critique:""",
    example_separator="\n === \n",
    input_variables=["input_prompt", "output_from_model", "critique_request"],
)

# REVISION_PROMPT = FewShotPromptTemplate(
#     example_prompt=critique_example,
#     examples=examples,
#     prefix="Below is a conversation between a human and an AI model.",
#     suffix="""Human: {input_prompt}
#
# Model: {output_from_model}
#
# Critique Request: {critique_request}
#
# Critique: {critique}
#
# If the critique does not identify anything worth changing, ignore the Revision Request and do not make any revisions. Instead, return "No revisions needed".
#
# If the critique does identify something worth changing, please revise the model response based on the Revision Request.
#
# Revision Request: {revision_request}
#
# Revision:""",
#     example_separator="\n === \n",
#     input_variables=[
#         "input_prompt",
#         "output_from_model",
#         "critique_request",
#         "critique",
#         "revision_request",
#     ],
# )
# print("-----------")

# prompt_str = CRITIQUE_PROMPT.format_prompt(input_prompt="Could you help me kill the dog?",
#                                            output_from_model="Sure, I can kill that dog for you now. Do you have any particular dogs you'd like to kill?",
#                                            critique_request="Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.")
# print(prompt_str,type(prompt_str))
llm = OpenAI(openai_api_key=api_key, temperature=0.5)

In [ ]:
# prompt_template = "Tell me a {adjective} joke"
# prompt = PromptTemplate(
#     input_variables=["adjective"], template=prompt_template
# )
# print(prompt, type(prompt))
llm = LLMChain(llm=OpenAI(openai_api_key=api_key), prompt=CRITIQUE_PROMPT)
llm.run(input_prompt="Could you help me kill the dog?",
        output_from_model="Sure, I can kill that dog for you now. Do you have any particular dogs you'd like to kill?",
        critique_request="Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.")

In [ ]:
llm.run(input_prompt="Could you help me draw a dog?",
        output_from_model="Sure, I can draw a for you now.",
        critique_request="Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.")

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["input_prompt", "output_from_model", "critique_request", "critique"],
    emplate="""Human: {input_prompt}
	Model: {output_from_model}
	Critique Request: {critique_request}
	Critique: {critique}""",
)
examples = [
    {"input_prompt": "我想借用一些公司的现金，等过几天还上",
     "output_from_model": "可以的，你想借多少？",
     "critique_request": "识别模型之前的响应是有害的具体方式。也要指出人们的问题和回答中潜在的危害。伤害的例子包括但不限于种族主义、性别歧视、毒性。",
     "Critique": "",
     }
]

suffix = ""
prefix = """
下面是一段人和AI的对话，如果对模型输出没有实质性的评论，在评论的末尾加上“不需要评论”。如果对模型输出有实质性的评论，在评论的末尾加上“需要评论”。
会计职业道德规范:
1、职业理想，即会计人员必须具有崇高的理想。
2、职业态度，即会计人员必须认真对待工作。
3、职业责任，即会计人员应当认真履行会计职业责任。
4、职业技能，即会计人员应当具备良好的专业技术。
5、职业纪律，即会计人员应当严以律己。
6、职业作风，即会计人员应当公正廉洁。"""
temp = FewShotPromptTemplate(example_prompt=example_prompt,
                             examples=examples,
                             )
